In [15]:
import pandas as pd
import numpy as np
df = pd.read_csv('../data/01_data.csv')  
df.head()

,district,area,bedrooms,garage,type,rent
0,Belenzinho,21,1,0,Studio e kitnet,2400
1,Vila Marieta,15,1,1,Studio e kitnet,1030
2,Pinheiros,18,1,0,Apartamento,4000
3,Vila Ré,56,2,2,Casa em condomínio,1750
4,Bela Vista,19,1,0,Studio e kitnet,4000


In [16]:
df_encoded = df.copy()
df_encoded['fold'] = np.random.randint(0, 5, size=len(df))  # 5 folds

# Configurações
cat_cols = ['district', 'type']
target = 'rent'
n_folds = 5
min_samples = 100  # valor de regularização: quanto maior, mais peso na média global

for col in cat_cols:
    encoded_col = f'{col}_te'  # Nome da nova coluna
    df_encoded[encoded_col] = np.nan  # Inicializa com NaN
    
    for fold in range(n_folds):
        # Dados de treino e validação
        train = df_encoded[df_encoded['fold'] != fold]
        val = df_encoded[df_encoded['fold'] == fold]
        
        # Cálculo do smoothing
        category_stats = train.groupby(col)[target].agg(['mean', 'count'])
        global_mean = train[target].mean()

        # Média suavizada
        smoothing = (category_stats['mean'] * category_stats['count'] + global_mean * min_samples) / (category_stats['count'] + min_samples)
        
        # Mapeia os valores suavizados no fold de validação
        val_encoded = val[col].map(smoothing)
        df_encoded.loc[val.index, encoded_col] = val_encoded

    # Para categorias nunca vistas (NaN), preenche com média global final
    final_global_mean = df_encoded[target].mean()
    df_encoded[encoded_col] = df_encoded[encoded_col].fillna(final_global_mean)

# Remove colunas originais e auxiliar
df_encoded.drop(columns=cat_cols + ['fold'], inplace=True)

# Resultado
print(df_encoded.columns.tolist())
df_encoded.head()


['area', 'bedrooms', 'garage', 'rent', 'district_te', 'type_te']


,area,bedrooms,garage,rent,district_te,type_te
0,21,1,0,2400,3192.638775,2242.105525
1,15,1,1,1030,3189.317811,2225.616343
2,18,1,0,4000,3975.327226,3348.582028
3,56,2,2,1750,3137.082294,3635.010525
4,19,1,0,4000,2849.582789,2242.105525


### Explicação

Este código implementa o **Target Encoding** com validação cruzada **K-fold** para transformar variáveis categóricas em valores numéricos baseados na média da variável alvo (`rent`), evitando vazamento de dados.

#### Lógica:
1. **Cópia dos Dados Originais**
   Preserva o `DataFrame` original em `df_encoded`.

2. **Criação de Folds Aleatórios**
   Divide o conjunto em 5 grupos (`fold`) aleatórios para validação cruzada.

3. **Configuração de Colunas e Parâmetros**
   Define as colunas categóricas (`district`, `type`) e o alvo (`rent`).
   Define `min_samples = 100` como o fator de suavização.

4. **Loop por Coluna Categórica**
   Para cada coluna categórica:

   * Cria uma nova coluna com sufixo `_te` (ex: `district_te`).
   * Inicializa com `NaN`.

5. **Loop por Fold (Validação Cruzada)**
   Para cada fold:

   * Divide os dados em **treino** e **validação**.

   * Calcula:

     * A média da variável alvo (`rent`) por categoria.
     * A contagem de amostras por categoria.
     * A média global da variável alvo no conjunto de treino.

   * Aplica a **fórmula de suavização**.

   * Substitui os valores da coluna categórica pela média suavizada apenas no fold de validação (evitando vazamento).

6. **Tratamento de Categorias Ausentes**
   Se alguma categoria estiver ausente em todos os folds de treino, preenche com a média global final.

7. **Limpeza do DataFrame**
   Remove as colunas categóricas originais e a coluna `fold`.


#### Objetivo:
O objetivo é realizar o **Target Encoding** de forma robusta, utilizando a técnica de validação cruzada K-fold e Suavização para evitar vazamento de dados. Isso garante que as médias calculadas para cada categoria sejam baseadas apenas nos dados de treino, sem incluir informações do fold de validação. O resultado é um conjunto de dados preparado para modelos de aprendizado de máquina.

#### Técnica K-fold:
A validação cruzada K-fold divide os dados em `n` subconjuntos (folds). Em cada iteração, um fold é usado como validação, enquanto os outros `n-1` são usados como treino. Isso permite que o modelo seja avaliado em diferentes partes dos dados, reduzindo o risco de overfitting e garantindo maior generalização. No contexto deste código, o K-fold é usado para calcular médias de forma isolada para cada fold, evitando que os dados de validação influenciem os cálculos. 

#### Técnica de Suavização:

A **suavização** evita que categorias com poucas observações tenham influência desproporcional. Exemplo:

* Uma categoria com **1 ocorrência** e valor `20000` teria média `20000` sem suavização.
* Com suavização, ela é "puxada" para mais perto da média global, gerando encoding mais estável.

* A fórmula de suavização:

     $$
     \text{encoding final} = \frac{\mu_{\text{categoria}} \cdot n + \mu_{\text{global}} \cdot m}{n + m}
     $$

     Onde:

     * $\mu_{\text{categoria}}$: média do `rent` da categoria
     * $n$: número de ocorrências da categoria
     * $\mu_{\text{global}}$: média global do `rent`
     * $m$: parâmetro de suavização (`min_samples`)


In [17]:
df_noisy = df_encoded.copy()

encoded_cols = ['district_te', 'type_te']

# Parâmetro do ruído (1% de variação)
noise_level = 0.01 

# Aplica ruído aleatório
for col in encoded_cols:
    noise = np.random.normal(1.0, noise_level, size=len(df_noisy))
    df_noisy[col] = df_noisy[col] * noise

df_noisy.head()

,area,bedrooms,garage,rent,district_te,type_te
0,21,1,0,2400,3208.377274,2214.082544
1,15,1,1,1030,3177.589720,2200.501677
2,18,1,0,4000,4052.823895,3334.700747
3,56,2,2,1750,3155.789213,3591.190504
4,19,1,0,4000,2839.546630,2241.411678


### Adição de Ruído Aleatório nas Colunas Codificadas

Após a aplicação do Target Encoding, é possível que os valores codificados fiquem altamente correlacionados com a variável alvo, o que pode levar ao overfitting. Para mitigar esse risco, adicionamos um **pequeno ruído aleatório multiplicativo** nas colunas codificadas.

#### Objetivo:
Evitar que o modelo aprenda valores codificados exatos e tornar o modelo mais robusto à variação dos dados.

#### Como funciona:
- Para cada valor codificado (ex: `district_te`, `type_te`), multiplicamos por um fator aleatório normalmente distribuído em torno de 1.0.
- Exemplo: um valor de 2150 pode se tornar 2135 ou 2168, dependendo do ruído.

#### Parâmetro:
- `noise_level = 0.01`: Define a intensidade do ruído (1%). Pode ser ajustado conforme a sensibilidade do modelo.

#### Resultado:
O DataFrame `df_noisy` contém os valores codificados com um leve ruído, prontos para serem usados em modelos de machine learning.


In [18]:
df_noisy.to_csv('../data/02_data.csv', index=False)